# L2: Evaluate Inputs: Classification

- 이 파일에서는 LLM을 zero-shot learning을 통해 간단한 분류기로 사용하는 예시를 보여줌

## Setup


In [ ]:
# 구글 코립일 경우
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.1 MB/s eta 0

In [ ]:
import os
import openai
import tiktoken

from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

#### helper function


In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens, # 모델이 출력하는 최대 토큰
    )

    return response.choices[0].message.content


#### 고객의 문의를 적절한 카테고리로 분류 하기

- 문의를 분류 하고 분류에 따라 적합하기 따로 개발된 프롬프트로 고객 응대를 하는 것이 응답의 정확성을 높일 수 있음

In [ ]:
delimiter = "####"

system_message = f"""
고객 서비스 문의 사항이 제공됩니다.
고객 서비스 쿼리는 {delimiter} 문자로 구분됩니다.
각 쿼리를 기본 카테고리와 보조 카테고리로 분류합니다.
기본 및 보조 키가 포함된 json 형식의 출력을 제공합니다.

기본 카테고리: 요금청구, 기술지원, 계정관리 또는 일반문의.

요금청구에 대한 보조 카테고리:
구독 취소 또는 업그레이드
결제 방법 추가
청구에 대한 설명
청구에 대한 이의 제기

기술지원에 대한 보조 카테고리
일반적인 문제 해결
장치 호환성
소프트웨어 업데이트

계정관리에 대한 보조 카테고리
비밀번호 재설정
개인 정보 업데이트
계정 폐쇄
계정 보안

일반문의에 대한 보조 카테고리:
제품 정보
가격
피드백
사람과 대화하기
"""
user_message = f"""
내 프로필과 사용자 데이터를 모두 삭제하고 싶습니다."""

messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]

response = get_completion_from_messages(messages)
print(response)

기본 카테고리: 계정관리
보조 카테고리: 계정 폐쇄

{
  "기본 카테고리": "계정관리",
  "보조 카테고리": "계정 폐쇄"
}


In [ ]:
user_message = f"""평면 스크린 TV에 대해서 알고 싶습니다."""

messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]

response = get_completion_from_messages(messages)
print(response)

{
  "기본 카테고리": "일반문의",
  "보조 카테고리": "제품 정보"
}
